In [1]:
import Formatting

include("jlFiles/printmat.jl")
include("jlFiles/NWFn.jl")
include("jlFiles/SRFn.jl")

SRFn (generic function with 1 method)

In [2]:
xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
x    = xx[1]
Rme  = x[:,2]/100
T    = size(Rme,1)

388

In [3]:
mu    = mean(Rme,1)                  #GMM to estimate mean and 2nd moment
mu_2  = mean(Rme.^2,1)
println("\n","[mu mu_2]")
printmat([mu mu_2])

g    = [(Rme .- mu) (Rme.^2 .- mu_2)]    #moment conditions
T    = size(g,1)
gbar = mean(g,1)
println("Sample moment conditions, gbar ")
printmat(gbar)

S = NWFn(g,1)                     #Var[sqrt(T)*gbar], Newey-West
D = -1
V = inv(D*inv(S)*D')              #Var[sqrt(T)*(mu,mu_2)]
println("Cov(params)")
printmat(V,10,6)


[mu mu_2]
     0.006     0.002

Sample moment conditions, gbar 
     0.000    -0.000

Cov(params)
  0.002317 -0.000077
 -0.000077  0.000020



In [4]:
                                  #delta method
df = [ (mu_2/(mu_2 - mu.^2).^(3/2)) (-mu/(2*(mu_2 - mu.^2).^(3/2)))]
println("\nAnalytical derivatives ")
printmat(df)

par0 = [mu;mu_2]
SR0  = SRFn(par0)       #Sharpe ratio, function

Delta = 1e-7            #numerical derivatives
k     = 2               #number of parameters
Jac   = fill(NaN,(1,k))
for j = 1:k              #loop over columns (parameters)
  btilde    = par0 + 0.0 #break the link between par0 and btilde, now not identical
  btilde[j] = btilde[j] + Delta
  Jac[1,j]  = (SRFn(btilde)- SR0)/Delta
end
println("\nNumerical derivatives, just for comparison ")
printmat(Jac)

Std_SR = sqrt(df*V*df'/T)
println("\nSR and its Std ")
printmat([SR0 Std_SR])

println("SR and 90% conf band: ")
printmat([SR0 (SR0-1.65*Std_SR) (SR0+1.65*Std_SR)])


Analytical derivatives 
    22.121   -30.957


Numerical derivatives, just for comparison 
    22.121   -30.956


SR and its Std 
     0.131     0.057

SR and 90% conf band: 
     0.131     0.037     0.225

